<a href="https://colab.research.google.com/github/ManasiBorhade/Multiple-Disease-Prediction/blob/main/kerastuner_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras-tuner

     |████████████████████████████████| 71kB 5.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=ceca4df3d247f0277bc80bc8edc97fc7b1c2d227a41a2210191683db4457b62d
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c1140df9a0670936bab34e3998d8b6e6c3afce9bd7cd467648a23258817d6aa6
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
train_images=train_images/255.0
test_images=test_images/255.0
train_images[0].shape
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

def hyper_model(hp):  
  from keras.models import Sequential
  from keras.layers import Conv2D
  from keras.layers import Dropout,Flatten,Dense
  model = Sequential()
  model.add(Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1))),
  model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu')),
  model.add(Flatten()),
  model.add(Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu')),
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model



In [ ]:
import kerastuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
tunersearch=RandomSearch(hyper_model,objective='val_accuracy',max_trials=5)
tunersearch.search(train_images,train_labels,epochs=3,validation_split=0.1)
model=tunersearch.get_best_models(num_models=1)[0]
model.summary()

Trial 3 Complete [00h 23m 55s]
val_accuracy: 0.9129999876022339

Best val_accuracy So Far: 0.9129999876022339
Total elapsed time: 01h 14m 35s
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                743456    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 840,634
Trainable params: 840,634
Non-trainable params: 0
__